<div style="background-color: #f0f8ff; padding: 15px; border-radius: 5px; border-left: 5px solid #4169e1;">
<h2 style="color: #4169e1;">Project Overview</h2>
<p style="color: #333;">In this project, I aim to develop a model capable of generating song lyrics using Recurrent Neural Networks (RNNs). The objective is to explore the creative potential of RNNs in text generation, specifically within the context of songwriting. Building a lyrics generator has long been on my list of projects, and I am excited to finally bring this idea to life by leveraging deep learning techniques to produce original and creative text.</p>
</div>

<div style="background-color: #e6ffe6; padding: 10px; border-radius: 5px; border-left: 5px solid #228B22;">
<h2 style="color: #228B22;">Importing Libraries</h2>
</div>

In [4]:
# pip install tensorflow

In [5]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.tokenize import word_tokenize

nltk.download("punkt")
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle
import warnings

warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load Dataset

In [6]:
data = pd.read_csv("spotify_songs.csv")

data.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


### To keep the model simple I am going to drop unneccesary columns. I will use track_name , trac_artist and lyrics columns

In [7]:
data = data[["track_name", "track_artist", "lyrics"]].rename(
    columns={"track_name": "Song_Title", "track_artist": "Artist", "lyrics": "Lyrics"}
)

In [8]:
data.head()

,Song_Title,Artist,Lyrics
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu..."
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U..."
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...


In [9]:
data.isnull().sum()

,0
Song_Title,0
Artist,0
Lyrics,260


In [10]:
# filling missing values with empty string
data["Lyrics"] = data["Lyrics"].fillna("")
data = data[data["Lyrics"].str.strip() != ""]
data = data.reset_index(drop=True)

In [11]:
data.isnull().sum()

,0
Song_Title,0
Artist,0
Lyrics,0


In [12]:
# print the artists in the data
print("Artists in the data:\n",data.Artist.value_counts())

Artists in the data:
 Artist
Queen                 123
Don Omar               74
David Guetta           73
Drake                  65
Guns N' Roses          63
                     ... 
Christina Grimmie       1
Luke Bryan              1
Magic City Hippies      1
Rocco Hunt              1
Steady Rollin           1
Name: count, Length: 5946, dtype: int64


In [13]:
data.shape

(18194, 3)

### Extracting more information on the songs such as:

* Number of characters
* Number of words
* Number of lines

In [14]:
#Adding a column of numbers of Characters,words and sentences in each msg
import nltk
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

data["No_of_Characters"] = data["Lyrics"].apply(len)
data["No_of_Words"]=data.apply(lambda row: nltk.word_tokenize(row["Lyrics"]), axis=1).apply(len)
data["No_of_Lines"] = data["Lyrics"].str.split('\n').apply(len)
display(data.describe())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,No_of_Characters,No_of_Words,No_of_Lines
count,18194.000000,18194.000000,18194.0
mean,2138.102067,510.558646,1.0
std,1713.122517,428.185625,0.0
min,4.000000,1.000000,1.0
25%,1164.000000,269.000000,1.0
50%,1693.000000,400.000000,1.0
75%,2572.000000,617.750000,1.0
max,27698.000000,6748.000000,1.0


### Removing very short and very long lyrics (e.g., less than 20 words or more than 2000 words).
#### Removing duplicates and empty lyrics.

In [15]:
# Remove very short or very long lyrics
data = data[(data['Lyrics'].str.split().apply(len) > 20) & (data['Lyrics'].str.split().apply(len) < 2000)]
data = data.drop_duplicates(subset=['Lyrics'])
data = data[data['Lyrics'].str.strip() != '']
data = data.reset_index(drop=True)

### Data Prepration

In [16]:
    from tensorflow.keras.preprocessing.text import Tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data['Lyrics'])

In [23]:
MAX_SEQ_LEN = 40

sequences = []
for lyric in data['Lyrics']:
    token_list = tokenizer.texts_to_sequences([lyric])[0]
    # Generate all possible n-grams up to MAX_SEQ_LEN
    n_grams = [token_list[:i+1] for i in range(1, min(len(token_list), MAX_SEQ_LEN))]
    sequences.extend(n_grams)

sequences = pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='pre')
X, y = sequences[:,:-1], sequences[:,-1]

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, 100, input_length=MAX_SEQ_LEN-1))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [19]:
# import tensorflow as tf
# # print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# # print("TensorFlow version:", tf.__version__)

In [20]:
  # !nvidia-smi

### Model Training

In [27]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, start_from_epoch=10),
    ModelCheckpoint('best_lyrics_model.keras', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    X, y,
    epochs=20,
    validation_split=0.1,
    batch_size=128,
    callbacks=callbacks
)

Epoch 1/20
4317/4317 ━━━━━━━━━━━━━━━━━━━━ 217s 50ms/step - accuracy: 0.1402 - loss: 5.1480 - val_accuracy: 0.1253 - val_loss: 6.2413
Epoch 2/20
4317/4317 ━━━━━━━━━━━━━━━━━━━━ 258s 49ms/step - accuracy: 0.1516 - loss: 4.9016 - val_accuracy: 0.1286 - val_loss: 6.3188
Epoch 3/20
4317/4317 ━━━━━━━━━━━━━━━━━━━━ 271s 52ms/step - accuracy: 0.1649 - loss: 4.7141 - val_accuracy: 0.1339 - val_loss: 6.3586
Epoch 4/20
4317/4317 ━━━━━━━━━━━━━━━━━━━━ 253s 49ms/step - accuracy: 0.1783 - loss: 4.5403 - val_accuracy: 0.1343 - val_loss: 6.4165
Epoch 5/20
4317/4317 ━━━━━━━━━━━━━━━━━━━━ 271s 52ms/step - accuracy: 0.1919 - loss: 4.3971 - val_accuracy: 0.1347 - val_loss: 6.5108
Epoch 6/20
4317/4317 ━━━━━━━━━━━━━━━━━━━━ 252s 49ms/step - accuracy: 0.2067 - loss: 4.2603 - val_accuracy: 0.1364 - val_loss: 6.5610
Epoch 7/20
4317/4317 ━━━━━━━━━━━━━━━━━━━━ 223s 52ms/step - accuracy: 0.2188 - loss: 4.1593 - val_accuracy: 0.1338 - val_loss: 6.6269
Epoch 8/20
4317/4317 ━━━━━━━━━━━━━━━━━━━━ 224s 52ms/step - accuracy: 

### Lyrics generation Funcion

In [28]:
import numpy as np

def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_lyrics(seed_text, next_words=50, max_seq_len=50, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)[0]
        output_index = sample_with_temperature(predicted, temperature)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == output_index:
                output_word = word
                break
        if output_word == "":
            break
        seed_text += " " + output_word
    return seed_text

### Saving Model

In [29]:
# Save model and tokenizer for deployment
model.save('lyrics_model.h5')
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)